## Count or frequency encoding - Feature-engine

[Feature Engineering for Machine Learning Course](https://www.trainindata.com/p/feature-engineering-for-machine-learning)

In count encoding we replace the categories by the count of the observations that show that category in the dataset. Similarly, we can replace the category by the frequency -or percentage- of observations in the dataset. That is, if 10 of our 100 observations show the colour blue, we would replace blue by 10 if doing count encoding, or by 0.1 if replacing by the frequency. These techniques capture the representation of each label in a dataset, but the encoding may not necessarily be predictive of the outcome. These are however, very popular encoding methods in Kaggle competitions.

The assumption of this technique is that the number observations shown by each variable is somewhat informative of the predictive power of the category.


### Advantages

- Simple
- Does not expand the feature space

### Disadvantages

- If 2 different categories appear the same amount of times in the dataset, that is, they appear in the same number of observations, they will be replaced by the same number: may lose valuable information.

For example, if there are 10 observations for the category blue and 10 observations for the category red, both will be replaced by 10, and therefore, after the encoding, will appear to be the same thing. 


Follow this [thread in Kaggle](https://www.kaggle.com/general/16927) for more information.



## In this demo:

We will see how to perform count or frequency encoding with Feature-Engine using the House Prices dataset.

For guidelines to obtain the dataset check **section 2** of the course.

In [8]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.4

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 81.5 MB/s eta 0:00:00:00:01:01


In [16]:
!pip install feature-engine

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

# to encode with feature-engine
from feature_engine.encoding import CountFrequencyEncoder

In [18]:
# load dataset

data = pd.read_csv(
    "/kaggle/input/housing/houseprice.csv",
    usecols=["Neighborhood", "Exterior1st", "Exterior2nd", "SalePrice"],
)

data.head()

,Neighborhood,Exterior1st,Exterior2nd,SalePrice
0,CollgCr,VinylSd,VinylSd,208500
1,Veenker,MetalSd,MetalSd,181500
2,CollgCr,VinylSd,VinylSd,223500
3,Crawfor,Wd Sdng,Wd Shng,140000
4,NoRidge,VinylSd,VinylSd,250000


In [19]:
# let's have a look at how many labels each variable has

for col in data.columns:
    print(col, ": ", len(data[col].unique()), " labels")

Neighborhood :  25  labels
Exterior1st :  15  labels
Exterior2nd :  16  labels
SalePrice :  663  labels


### Important

When doing count transformation of categorical variables, it is important to calculate the count (or frequency = count / total observations) **over the training set**, and then use those numbers to replace the labels in the test set.

In [20]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data[["Neighborhood", "Exterior1st", "Exterior2nd"]],  # predictors
    data["SalePrice"],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0,
)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((1022, 3), (438, 3))

## Count or Frequency Encoding with Feature-Engine

In [21]:
count_enc = CountFrequencyEncoder(
    encoding_method="count",  # to do frequency ==> encoding_method='frequency'
    variables=["Neighborhood", "Exterior1st", "Exterior2nd"],
)

count_enc.fit(X_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

CountFrequencyEncoder(variables=['Neighborhood', 'Exterior1st', 'Exterior2nd'])

In [6]:
# in the encoder dict we can observe the number of
# observations per category for each variable

count_enc.encoder_dict_

{'Neighborhood': {'NAmes': 151,
  'CollgCr': 105,
  'OldTown': 73,
  'Edwards': 71,
  'Sawyer': 61,
  'Somerst': 56,
  'Gilbert': 55,
  'NWAmes': 51,
  'NridgHt': 51,
  'SawyerW': 45,
  'BrkSide': 41,
  'Mitchel': 36,
  'Crawfor': 35,
  'Timber': 30,
  'NoRidge': 30,
  'ClearCr': 24,
  'IDOTRR': 24,
  'SWISU': 18,
  'StoneBr': 16,
  'Blmngtn': 12,
  'MeadowV': 12,
  'BrDale': 10,
  'NPkVill': 7,
  'Veenker': 6,
  'Blueste': 2},
 'Exterior1st': {'VinylSd': 364,
  'HdBoard': 153,
  'Wd Sdng': 148,
  'MetalSd': 138,
  'Plywood': 86,
  'CemntBd': 39,
  'BrkFace': 35,
  'WdShing': 21,
  'Stucco': 17,
  'AsbShng': 15,
  'Stone': 2,
  'AsphShn': 1,
  'BrkComm': 1,
  'ImStucc': 1,
  'CBlock': 1},
 'Exterior2nd': {'VinylSd': 353,
  'Wd Sdng': 142,
  'HdBoard': 141,
  'MetalSd': 136,
  'Plywood': 112,
  'CmentBd': 39,
  'Wd Shng': 29,
  'BrkFace': 18,
  'AsbShng': 17,
  'Stucco': 16,
  'ImStucc': 8,
  'Stone': 4,
  'Brk Cmn': 4,
  'AsphShn': 1,
  'CBlock': 1,
  'Other': 1}}

In [7]:
X_train = count_enc.transform(X_train)
X_test = count_enc.transform(X_test)

# let's explore the result
X_train.head()

,Neighborhood,Exterior1st,Exterior2nd
64,105,364,353
682,24,148,142
960,41,148,112
1384,71,21,29
1100,18,148,142


**Note**

If the argument variables is left to None, then the encoder will automatically identify all categorical variables. Is that not sweet?

The encoder will not encode numerical variables. So if some of your numerical variables are in fact categories, you will need to re-cast them as object before using the encoder.

Note, if there is a variable in the test set, for which the encoder doesn't have a number to assigned (the category was not seen in the train set), the encoder will return an error.